<a href="https://colab.research.google.com/github/Jojocko/NLP-projects-/blob/main/Suite_we_cbow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Datasets NLP/dataset_supplychain.csv')

Mounted at /content/drive


In [2]:
df = df.drop(['client', 'langage', 'reponse'], axis=1)
df.rename(columns={'Sentiment': 'sentiment', 'Commentaire': 'commentaire'}, inplace=True)
df['sentiment'] = df['sentiment'].replace({'__label__POSITIVE': 'positif', '__label__NEGATIVE': 'negatif', '__label__NEUTRAL': 'neutre'})
df['date'] = df['date'].fillna(method="ffill")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year # pour visualiser par année et non par jour/mois/année

In [3]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('french'))

jugement = {'très', 'extrêmement', 'particulièrement', 'exceptionnellement','tout à fait', 'absolument', 'complètement', 'entièrement', 'parfaitement', 'profondément', 'hautement', 'tout', 'plutôt', 'assez', 'bien', 'bon','vraiment', 'totalement', 'énormément', 'peu', 'moins'}
satisfaction = {'satisfait', 'content', 'heureux', 'ravi', 'enchanté', 'comblé', 'agréable', 'plaisant', 'positif', 'excellent', 'remarquable', 'exceptionnel', 'superbe', 'admirable', 'réjoui', 'gratifiant', 'récompensant', 'conquis', 'impressionné', 'élogieux'}
insatisfaction = {'insatisfait', 'mécontent', 'déçu', 'frustré', 'contrarié', 'désappointé', 'inacceptable', 'problématique', 'inadmissible', 'déplorable', 'lamentable', 'irrité', 'en colère', 'révolté', 'amère', 'négatif', 'critique', 'malheureux', 'peu convaincu', 'regrettable'}
company = {'Fnac', 'fnac', 'Amazon', 'amazon', 'CDiscount', 'cdiscount'}

stop_words.update(jugement, satisfaction, insatisfaction, company)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Traitement des données pour clustering des commentaires négatifs

import re
import unicodedata
from nltk.tokenize import word_tokenize

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip()) # lower capitalized letters + retire les espaces en début et fin de string (phrases)
    w = re.sub(r"([?.!,¿])", r" \1 ", w) # ajout d'espaces autour des ponctuations pour les séparer des mots. eg: "he is a boy." => "he is a boy .
    w = re.sub(r'[" "]+', " ", w) # remplacement des séquences d'espaces multiples par un seul espace
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w) # suppression de tout caractère qui n'est pas une lettre ou une ponctuation courante
    w = re.sub(r'\b\w{0,2}\b', '', w) # suppression des mots de moins de trois lettres.

    # remove stopword
    mots = word_tokenize(w.strip()) # tokénization en mots individuels
    mots = [mot for mot in mots if mot not in stop_words] # filtrage des mots vides
    return ' '.join(mots).strip() # reconstruction de la phrase sans les mots vides

df.cleaned_lemma = df.cleaned_lemma.apply(lambda x :preprocess_sentence(x))

In [5]:
# Création de l'ensemble de données pour archtecture CBOW

# Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000) # doc limité à 10000 mots
tokenizer.fit_on_texts(df.cleaned_lemma) # application du dic sur les commentaires

word2idx = tokenizer.word_index # stocke le dictionnaire de correspondance entre mots et index
idx2word = tokenizer.index_word # stocke le dictionnaire de correspondance entre index et mots
vocab_size = tokenizer.num_words # stocke la taille du dictionnaire

# Ensemble de données
import numpy as np

def sentenceToData(tokens, WINDOW_SIZE):
    context_size = WINDOW_SIZE // 2
    window = np.concatenate((np.arange(-context_size, 0), np.arange(1, context_size + 1)))
    X, Y = [], []
    for word_index, word in enumerate(tokens):
        if (word_index - context_size >= 0) and (word_index + context_size < len(tokens)):
            context = [tokens[word_index + offset] for offset in window]
            target = word
            X.append(context)
            Y.append(target)
    return X, Y

WINDOW_SIZE = 5  # Fenêtre de contexte autour du mot cible

X, Y = [], []
for review in df['cleaned_lemma'][df['sentiment'] == 'negatif']:
    sentences = review.split(".")  # transforme les commentaires en phrases
    for sentence in sentences:
        word_list = tokenizer.texts_to_sequences([sentence.strip()])[0]  # convertit les phrases en séquence
        if len(word_list) >= WINDOW_SIZE:
            X_temp, Y_temp = sentenceToData(word_list, WINDOW_SIZE)
            X.extend(X_temp)
            Y.extend(Y_temp)

# Convert X and Y to numpy arrays
X = np.array(X).astype(int)

from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X, padding='post')
Y = np.array(Y).reshape(-1, 1)

print('Shape of X:', X.shape)
print('Shape of Y:', Y.shape)


Shape of X: (764323, 4)
Shape of Y: (764323, 1)


In [6]:
# Architecture CBOW

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


vocab_size = min(len(word2idx) + 1, tokenizer.num_words + 1)
embedding_dim = 512

optimizer = Adam(learning_rate=0.001)

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
embedding_layer.trainable = False


model = Sequential()
model.add(embedding_layer)
model.add(GRU(units=128, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X, Y, epochs=20, batch_size=64, callbacks=[early_stopping], validation_split=0.1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         5120512   
                                                                 
 gru (GRU)                   (None, 128)               246528    
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10001)             2570257   
                                                                 
Total params: 7970321 (30.40 MB)
Trainable params: 2849809 (10.87 MB)
Non-trainable params: 5120512 (19.53 MB)
_________________________________________________________________
Epoch 1/20


In [7]:
# Sauvegarde du CBOW

model.save_weights('/content/drive/My Drive/weights.h5')

In [8]:
# Création de clusters bi-grammes

import numpy as np
from nltk import ngrams
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import text_to_word_sequence

negative_comments = df[df['sentiment'] == 'negatif']['cleaned_lemma']

# Extraction des embeddings
embeddings = model.layers[0].get_weights()[0]

n = 2  # bigrammes
n_gram_list = []
for comment in negative_comments:
    words = text_to_word_sequence(comment) # tokenization
    n_grams = list(ngrams(words, n))
    n_gram_list.extend(n_grams)

# Représentation vectorielle pour chaque bigramme
n_gram_vectors = []
for n_gram in n_gram_list:
    indices = [tokenizer.word_index.get(word, 0) for word in n_gram]
    # Checker si les indices sont valides (< vocab_size)
    if all(idx < vocab_size for idx in indices):
        word_vectors = embeddings[indices]
        n_gram_vector = np.mean(word_vectors, axis=0)
        n_gram_vectors.append(n_gram_vector)

# Standardisation des vecteurs de bigrammes
scaler = StandardScaler()
n_gram_vectors_scaled = scaler.fit_transform(n_gram_vectors)


In [17]:
# PCA pour réduire la dimensionnalité

from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Conserver 95% de la variance explicative
n_gram_vectors_pca = pca.fit_transform(n_gram_vectors_scaled)

# Clustering avec KMeans sur les données réduites
k = 5  # Nombre de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(n_gram_vectors_pca)
clusters = kmeans.labels_

# Affichage des clusters
cluster_n_grams = {i: [] for i in range(k)}
for i, label in enumerate(clusters):
    n_gram = n_gram_list[i]
    cluster_n_grams[label].append(" ".join(n_gram))

# Afficher les n-grammes pour chaque cluster
for cluster, n_grams in cluster_n_grams.items():
    print(f"Cluster {cluster}:")
    for n_gram in n_grams[:10]:  # Afficher les 10 premiers n-grammes pour concision
        print(n_gram)
    print("\n---\n")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0:
commander mardi
mardi fevrier
jeudi mars
non recu
recu mars
mars prix
incapable chronopost
pareilservic client
client veule
patient journee

---

Cluster 1:
fevrier livraison
livraison jeudi
chronopost pareilservic
retouramazon fuir
fuir dernier
vouloir acheter
acheter americain
dire signer
signer soin
gros objet

---

Cluster 2:
methode voleur
voleur voyou
telephone incompetent
incompetent version
recevoir service
service client
faible rapport
conforme legislation
impossible service
service client

---

Cluster 3:
mars produit
produit non
renvoyer article
article semaine
version jour
jour retouramazon
blanc jusque
jusque raccrocher
amazone voler
voler dernier

---

Cluster 4:
colis commander
prix colis
colis incapable
veule patient
balle amazonamazon
remboursement methode
raccrocher appel
saturer droite
droite faible
disque dur

---



In [23]:
# Définition des noms de clusters basés sur les thèmes identifiés
cluster_names = {
    0: "Problèmes de délai ou de réception",
    1: "Déception du service de livraison",
    2: "Critiques sévères du service client",
    3: "Retours et remboursements",
    4: "Produits défectueux et insatisfaction générale"
}

In [24]:
# Score de silhouette

# Calcul long d'où échantillonnage des données

from sklearn.metrics import silhouette_score

# Calculer le score de silhouette avec un échantillon limité
score = silhouette_score(n_gram_vectors_pca, kmeans.labels_, sample_size=50000)
print(f"Score de silhouette (limité): {score}")


Score de silhouette (limité): 0.02242829462556348


In [25]:
# Test des clusters formés

ref_df = pd.read_csv('/content/drive/My Drive/Datasets NLP/ecommerce_plaintes.csv')

# Accès aux commentaires
reference_sentences = ref_df['commentaire'].tolist()

# Tokenisation et conversion en embeddings pour les phrases de référence
ref_n_gram_vectors = []

for sentence in reference_sentences:
    words = text_to_word_sequence(sentence)
    n_grams = list(ngrams(words, n))  # n étant bigramme
    sentence_vectors = []
    for n_gram in n_grams:
        indices = [tokenizer.word_index.get(word, 0) for word in n_gram]
        if all(idx < vocab_size for idx in indices):
            word_vectors = embeddings[indices]
            n_gram_vector = np.mean(word_vectors, axis=0)
            sentence_vectors.append(n_gram_vector)
    # Moyenne des embeddings des n-grammes pour obtenir un vecteur par phrase
    if sentence_vectors:
        ref_n_gram_vectors.append(np.mean(sentence_vectors, axis=0))

ref_n_gram_vectors = np.array(ref_n_gram_vectors)


In [26]:
# Même objet PCA que pour les données d'entraînement

ref_n_gram_vectors_pca = pca.transform(ref_n_gram_vectors)

In [27]:
# Calcul de similarité

from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité de chaque phrase de référence avec les centroïdes des clusters
similarities = cosine_similarity(ref_n_gram_vectors_pca, kmeans.cluster_centers_)

# Pour chaque phrase, identification du cluster le plus proche
closest_clusters = np.argmax(similarities, axis=1)

for i, cluster_num in enumerate(closest_clusters):
    print(f"\nPhrase '{reference_sentences[i]}' est la plus proche de la problématique '{cluster_names[cluster_num]}'\n")




Phrase 'J'ai attendu plus d'un mois pour recevoir ma commande, c'est inacceptable.' est la plus proche de la problématique 'Critiques sévères du service client'


Phrase 'Le suivi de mon colis indique qu'il a été livré, mais je n'ai rien reçu.' est la plus proche de la problématique 'Déception du service de livraison'


Phrase 'Ma commande est restée en statut 'expédiée' pendant trois semaines.' est la plus proche de la problématique 'Déception du service de livraison'


Phrase 'La livraison était prévue pour hier, mais je n'ai toujours pas mon colis.' est la plus proche de la problématique 'Déception du service de livraison'


Phrase 'Le délai de livraison promis n'a pas été respecté, je suis très déçu.' est la plus proche de la problématique 'Déception du service de livraison'


Phrase 'Mon article est arrivé cassé, comment cela a-t-il pu arriver?' est la plus proche de la problématique 'Déception du service de livraison'


Phrase 'Le produit reçu est endommagé, je demande un rembou

In [29]:
# Préparation des vecteurs de commentaires pour l'affichage des clusters par entreprise

for comment in df['cleaned_lemma'][df['sentiment'] == 'negatif']:
    words = text_to_word_sequence(comment)  # Tokenisation
    word_indices = [tokenizer.word_index[word] for word in words if word in tokenizer.word_index]
    # Filtrer les indices qui sont dans la plage valide
    valid_indices = [idx for idx in word_indices if idx < len(embeddings)]
    if valid_indices:
        comment_vector = np.mean([embeddings[idx] for idx in valid_indices], axis=0)
        comment_vectors.append(comment_vector)



In [34]:

valid_indices = []

# Génération des vecteurs de commentaires
comment_vectors = []
for i, comment in enumerate(df['cleaned_lemma'][df['sentiment'] == 'negatif']):
    words = text_to_word_sequence(comment)  # Tokenisation
    word_indices = [tokenizer.word_index.get(word, -1) for word in words]
    valid_word_indices = [idx for idx in word_indices if 0 <= idx < len(embeddings)]
    if valid_word_indices:
        comment_vector = np.mean([embeddings[idx] for idx in valid_word_indices], axis=0)
        comment_vectors.append(comment_vector)
        valid_indices.append(i)  # Stocker l'indice du commentaire valide

comment_vectors = np.array(comment_vectors)

# Standardisation des vecteurs de commentaires
scaler = StandardScaler()
comment_vectors_scaled = scaler.fit_transform(comment_vectors)

# Clustering avec KMeans
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
clusters = kmeans.fit_predict(comment_vectors_scaled)

# Association des clusters aux commentaires
df_negatif_valid = df[df['sentiment'] == 'negatif'].iloc[valid_indices].copy()
df_negatif_valid['cluster'] = clusters




In [35]:
# Analyse des clusters par entreprise
cluster_company_distribution = df_negatif_valid.groupby(['cluster', 'company']).size().unstack(fill_value=0)
print(cluster_company_distribution)


company  Amazon  CDiscount  Fnac
cluster                         
0           258        878   230
1            23        369    38
2           495       1423   394
3            13        361     1
4          1046        800   223
5           216        725    81
6          2850       7998  2941
7           688       1499   466
8           245       1100   255
9           317       1528   276
